In [2]:
import tkinter as tk
from tkinter import filedialog, ttk
from PIL import Image, ImageTk, ImageEnhance # Used for Tkinter display and PIL enhancement
import cv2
import numpy as np


# --- 1. CORE IMAGE PROCESSING FUNCTIONS ---

def adjust_gamma(image, gamma):
    """Applies gamma correction (brightness adjustment) using a LUT."""
    # The formula is: output = 255 * (input / 255)^(1/gamma)
    if gamma == 0:
        return image
        
    invGamma = 1.0 / gamma
    # Create a lookup table
    table = np.array([((i / 255.0) ** invGamma) * 255
                      for i in np.arange(0, 256)]).astype("uint8")
    
    # Apply the lookup table
    return cv2.LUT(image, table)

def adjust_contrast(image, alpha):
    """Adjusts contrast using a simple linear function (alpha*I + beta)."""
    # Contrast adjustment formula: output = alpha * input + beta (where beta=0 for simplicity)
    # alpha=1 is original contrast. alpha>1 increases contrast. alpha<1 decreases contrast.
    
    # Using cv2.addWeighted for linear transformation, but simplified for contrast:
    return cv2.convertScaleAbs(image, alpha=alpha, beta=0)

# --- 2. TKINTER APPLICATION CLASS ---

class ImageAdjusterApp:
    def __init__(self, master):
        self.master = master
        master.title("Image app to adjust brightness and contrast")

        # Variables for image and processing
        self.original_cv_image = None
        self.current_cv_image = None
        self.tk_image = None
        
        # Tkinter variables for sliders
        self.brightness_val = tk.DoubleVar(value=1.0)
        self.contrast_val = tk.DoubleVar(value=1.0)
        
        # --- GUI Setup ---
        self.setup_ui()

    def setup_ui(self):
        # Frame for Sliders and Controls
        control_frame = ttk.Frame(self.master, padding="10")
        control_frame.pack(side=tk.LEFT, fill=tk.Y)

        # Frame for Image Display
        self.image_frame = ttk.Label(self.master, relief="sunken")
        self.image_frame.pack(side=tk.RIGHT, expand=True, fill=tk.BOTH)

        # 1. Load Button
        ttk.Button(control_frame, text="Load Image", command=self.load_image).pack(fill=tk.X, pady=5)
        
        # 2. Brightness Slider (Gamma Correction)
        ttk.Label(control_frame, text="Brightness (Gamma):").pack(pady=(10, 0))
        ttk.Scale(
            control_frame,
            from_=0.1, to=3.0, # Gamma range
            orient=tk.HORIZONTAL,
            variable=self.brightness_val,
            command=self.update_image,
            length=200
        ).pack()

        # 3. Contrast Slider
        ttk.Label(control_frame, text="Contrast (Alpha):").pack(pady=(10, 0))
        ttk.Scale(
            control_frame,
            from_=0.1, to=3.0, # Alpha range
            orient=tk.HORIZONTAL,
            variable=self.contrast_val,
            command=self.update_image,
            length=200
        ).pack()
        
        # 4. Reset Button
        ttk.Button(control_frame, text="Reset Values", command=self.reset_values).pack(fill=tk.X, pady=15)

        # 5. Save Button
        ttk.Button(control_frame, text="Save Image", command=self.save_image).pack(fill=tk.X, pady=5)

        
    def load_image(self):
        """Opens a file dialog, loads the image, and displays it."""
        file_path = filedialog.askopenfilename(
            filetypes=[("Image Files", "*.jpg;*.jpeg;*.png;*.bmp")]
        )
        if not file_path:
            return

        # Load the image using OpenCV
        img = cv2.imread(file_path)
        if img is None:
            return
            
        # Store the original image (BGR format)
        self.original_cv_image = img
        
        # Reset sliders to initial state (1.0 for both)
        self.reset_values() 
        
        # Display the loaded image
        self.display_image(self.original_cv_image)

    def update_image(self, event=None):
        """Applies both brightness and contrast adjustments and updates the display."""
        if self.original_cv_image is None:
            return

        # Get current slider values
        gamma = self.brightness_val.get()
        alpha = self.contrast_val.get()

        # 1. Apply Contrast (Alpha) adjustment first
        contrast_adjusted = adjust_contrast(self.original_cv_image, alpha)
        
        # 2. Apply Brightness (Gamma) adjustment to the contrast-adjusted image
        final_image = adjust_gamma(contrast_adjusted, gamma)
        
        self.current_cv_image = final_image
        self.display_image(final_image)

    def display_image(self, cv_image):
        """Converts the OpenCV image (NumPy array) to a Tkinter PhotoImage and displays it."""
        
        # Convert BGR (OpenCV) to RGB (Pillow)
        cv_image_rgb = cv2.cvtColor(cv_image, cv2.COLOR_BGR2RGB)
        
        # Convert NumPy array to PIL Image
        pil_image = Image.fromarray(cv_image_rgb)
        
        # Determine the maximum size for display (e.g., 600x600)
        max_size = (600, 600)
        pil_image.thumbnail(max_size) 

        # Convert PIL Image to Tkinter PhotoImage
        self.tk_image = ImageTk.PhotoImage(pil_image)
        
        # Update the Label (image_frame) with the new image
        self.image_frame.config(image=self.tk_image)
        self.image_frame.image = self.tk_image # Keep a reference!

    def reset_values(self):
        """Resets sliders to their default values and updates the image."""
        self.brightness_val.set(1.0)
        self.contrast_val.set(1.0)
        self.update_image()
        
    def save_image(self):
        """Saves the currently displayed image."""
        if self.current_cv_image is None:
            return

        file_path = filedialog.asksaveasfilename(
            defaultextension=".png",
            filetypes=[("PNG file", "*.png"), ("JPEG file", "*.jpg")]
        )
        
        if file_path:
            # Save the processed image using OpenCV
            cv2.imwrite(file_path, self.current_cv_image)
            print(f"Image successfully saved to: {file_path}")

# --- 3. MAIN EXECUTION BLOCK ---

if __name__ == "__main__":
    root = tk.Tk()
    app = ImageAdjusterApp(root)
    root.mainloop()

Image successfully saved to: D:/Computer Vision Materials/Practice/output.png
